From https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
print(torch.__version__)
print(torch.cuda.is_available())
x = torch.rand(5, 3)
print(x)

1.13.0+cpu
False
tensor([[0.5213, 0.5965, 0.5369],
        [0.9415, 0.9284, 0.6731],
        [0.8012, 0.1481, 0.9731],
        [0.0372, 0.1861, 0.8291],
        [0.0315, 0.9544, 0.5621]])


In [15]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [16]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

batches = 0
for X, y in test_dataloader:
    if batches == 0:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
    batches += 1

print(f'{batches} in test db')

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
157 in test db
Using cpu device


In [39]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [45]:
model = NeuralNetwork().to(device)
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=2e-3)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
  )
)


In [46]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [47]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 6.260674  [    0/60000]
loss: 6.211079  [ 6400/60000]
loss: 6.140468  [12800/60000]
loss: 6.063318  [19200/60000]
loss: 5.889508  [25600/60000]
loss: 5.335482  [32000/60000]
loss: 3.801413  [38400/60000]
loss: 2.638418  [44800/60000]
loss: 2.440973  [51200/60000]
loss: 2.299294  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.290302 

Epoch 2
-------------------------------
loss: 2.375528  [    0/60000]
loss: 2.301287  [ 6400/60000]
loss: 2.213196  [12800/60000]
loss: 2.184240  [19200/60000]
loss: 2.079535  [25600/60000]
loss: 2.000661  [32000/60000]
loss: 2.017941  [38400/60000]
loss: 1.838941  [44800/60000]
loss: 1.843631  [51200/60000]
loss: 1.697118  [57600/60000]
Test Error: 
 Accuracy: 34.8%, Avg loss: 1.690951 

Epoch 3
-------------------------------
loss: 1.789171  [    0/60000]
loss: 1.669247  [ 6400/60000]
loss: 1.516152  [12800/60000]
loss: 1.510658  [19200/60000]
loss: 1.366234  [25600/60000]
loss: 1.378417  [32000/600

In [48]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
correct = 0
for i in range(100):
    x, y = test_data[i][0], test_data[i][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')
        if predicted == actual:
            correct += 1
print(correct)

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Shirt", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Coat"
Predicted: "Coat", Actual: "Shirt"
Predicted: "Sandal", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Coat", Actual: "Coat"
Predicted: "Sandal", Actual: "Sandal"
Predicted: "Sandal", Actual: "Sneaker"
Predicted: "Dress", Actual: "Dress"
Predicted: "Coat", Actual: "Coat"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Shirt", Actual: "Coat"
Predicted: "Bag", Actual: "Bag"
Predicted: "T-shirt/top", Actual: "T-shirt/top"
Predicted: "Shirt", Actual: "Pullover"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"
Predicted: "Sneaker", Actual: "Ankle boot"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Act